In [ ]:
## colab 파일 관리

# 1. 외부 .ipynb의 mimeType = "json"
# 2. 메뉴의 드라이브 마운트 기능은 그 드라이브에서 생성된 colab 파일에만 적용된다.
# 3. 드라이브 마운트 기능을 사용하려면, 


In [ ]:
import os
import sys
import io
from glob import glob
from pprint import pprint

## Google
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from googleapiclient.http import MediaIoBaseUpload

sys.path.insert(0, '/content/drive/MyDrive/__COLAB/PACKAGES')
import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈

sys.path.insert(0, '/content/drive/MyDrive/__COLAB/__GOOGLE')
## NOTE: 현재 디렉토리로 이동한 후 import해야 함
%cd /content/drive/MyDrive/__COLAB/__GOOGLE
from GoogleAuth import GoogleAuth
# from GoogleRestAPI import GoogleRestAPI
# from GoogleDrive.GoogleDrive import GoogleDrive

##
PATH_COLAB = "/content/drive/MyDrive/__COLAB"


class GoogleColab():
    def __init__(self, user_nick=None, mount=True):
        self.user_nick = user_nick
        self.gauth = None
        self.mounted = False
        self.drive = None

        if mount:  ## drive mount 시도 여부
            try:
                os.chdir(PATH_COLAB)
                self.mounted = True
            except:
                pass
       
        if user_nick:  ## client 계정 사용
            gauth = GoogleAuth(user_nick=user_nick)
            self.drive = build("drive", gauth.version["drive"], credentials = gauth.credentials)
    

    def list(self, path):
        if self.mounted:
            os.chdir(path)
            print(f"path: {path}")
            return glob("*.ipynb")
        elif self.drive:
            # path_to_id()
            self.drive.files()


    ## copy 이용: 드라이브 마운트 안됨!!!
    def _jupyter_to_colab(self, file_id):
        if not self.drive:
            print("Google Drive에 연결할 수 없습니다.")
            return None

        meta_data = self.drive.files().get(fileId=file_id).execute()
        if meta_data:
            name = meta_data['name']
            self.drive.files().copy(
                fileId = file_id,
                body = dict(
                    name = name,
                    mimeType = "application/vnd.google.colaboratory"
                )
            ).execute()

            self.drive.files().delete(fileId = file_id).execute()
        else:
            print("해당 파일이 없습니다.")
            return None


    # create 이용: 
    def _jupyter_to_colab(self, file_id):
        if not self.drive:
            print("Google Drive에 연결할 수 없습니다.")
            return None

        meta_data = self.drive.files().get(
            fileId = file_id,
            fields = "name, parents"
            # fields = "*"
        ).execute()

        pprint(meta_data)

        if meta_data:
            mime_type = "application/vnd.google.colaboratory"
            # mime_type = "application/x-ipynb+json"
            meta_data['name'] = "_" + meta_data['name']
            meta_data['originalFilename'] = meta_data['name']
            meta_data['mimeType'] = mime_type
            # media_body = MediaFileUpload(filename, mimetype=mime_type, resumable=True)  ## NOTE: Local File
            # media_body = MediaIoBaseUpload(io.BytesIO(json.dumps(content).encode('utf-8')), mime_type, resumable=True)  ## NOTE: string(text)
            media_body = MediaIoBaseUpload(io.BytesIO(self.drive.files().get_media(fileId=file_id).execute()), mime_type, resumable=True)  ## NOTE: Google Drive File

            # self.drive.files().delete(fileId = file_id).execute()
 
            return self.drive.files().create(
                body = meta_data,
                media_body = media_body
            ).execute()

        else:
            print("해당 파일이 없습니다.")
            return None
                

    # update 이용: 
    # def _jupyter_to_colab(self, file_id):
    #     if not self.drive:
    #         print("Google Drive에 연결할 수 없습니다.")
    #         return None

    #     meta_data = self.drive.files().get(
    #         fileId = file_id,
    #         fields = "name"
    #         # fields = "*"
    #     ).execute()

    #     # pprint(meta_data)

    #     if meta_data:
    #         mime_type = "application/vnd.google.colaboratory"  # 업로드시: 'mimeType': 'application/octet-stream'
    #         # mime_type = "application/json"  # "application/vnd.google.colaboratory"에서는 변경 안됨
    #         meta_data['mimeType'] = mime_type
    #         # meta_data['originalFilename'] = meta_data['name']

    #         return self.drive.files().update(
    #             fileId = file_id,
    #             body = meta_data
    #         ).execute()

    #     else:
    #         print("해당 파일이 없습니다.")
    #         return None



    def _colab_to_jupyter(self, file_id):
        pass
    

    def _colab_to_py(self, file_id, markdown=True):
        pass
    

    def _py_to_colab(self, file_id, markdown=True):
        pass
    

    def copy(self, src_path, dst_path, src_type, dst_type, move=False):
        pass


    def copy_all(self, src_folder, dst_folder, src_type, dst_type):
        pass


    def copy_to_all(self, src_folder, dst_folder, src_type, dst_type):
        pass


if __name__ == "__main__":
    ## list
    gc = GoogleColab(user_nick="satsbymoon")
    # file_id = "1V2eu8aE3RH7PpZsHD8nL-qKFqh6wvk3B"  ## 업로드 파일 'mimeType': 'application/octet-stream'
    file_id = "1ebHwh_UerlouFyWhoHa0N01b8D2tuDGH"  ## 업로드 파일 -> create 'application/x-ipynb+json'
    file_id = "1S7w0q1QFAHbWvMrmL2402JHkagVC5dPF"  ## application/x-ipynb+json -> application/vnd.google.colaboratory 1ebHwh_UerlouFyWhoHa0N01b8D2tuDGH
    # 'originalFilename': '' vs # 'originalFilename': 'GoogleColab.ipynb'
    # gc._jupyter_to_colab(file_id)  ## 업로드 파일 -> create 'application/json' -> update 'application/colab'
    r = gc.drive.files().get(fileId=file_id, fields="*").execute()
    pprint(r)

    # file_id = "11XdqELYDyDea_4YLmJHSjSrimazLdtr-"  ## colab 드라이브 마운트 가능
    
    # gc._jupyter_to_colab(file_id)
    # path = "/content/drive/MyDrive/__COLAB/__GOOGLE"
    # l = gc.list(path)
    # print(l)
    # m = gc.drive.files().get(fileId="11XdqELYDyDea_4YLmJHSjSrimazLdtr-").execute()
    # print(m)
    # file_id = "11XdqELYDyDea_4YLmJHSjSrimazLdtr-"
    # gc.drive.files().copy(
    #     fileId = file_id,
    #     name = name,
    # ).execute()
    # print(os.getcwd())
    # # print(os.path.dirname(__file__))
    # # /content/drive/MyDrive
    # try:
    #     os.chdir("/content/drive")
    #     mounted = True
    # except:
    #     mounted = False
    
    # print(f"mounted: {mounted}")


/content/drive/MyDrive/__COLAB/__GOOGLE
importing Jupyter notebook from GoogleAuth.ipynb
{'capabilities': {'canAcceptOwnership': False,
                  'canAddChildren': False,
                  'canAddMyDriveParent': False,
                  'canChangeCopyRequiresWriterPermission': True,
                  'canChangeSecurityUpdateEnabled': False,
                  'canChangeViewersCanCopyContent': True,
                  'canComment': True,
                  'canCopy': True,
                  'canDelete': True,
                  'canDownload': True,
                  'canEdit': True,
                  'canListChildren': False,
                  'canModifyContent': True,
                  'canMoveChildrenWithinDrive': False,
                  'canMoveItemIntoTeamDrive': True,
                  'canMoveItemOutOfDrive': True,
                  'canMoveItemWithinDrive': True,
                  'canReadRevisions': True,
                  'canRemoveChildren': False,
                  'canR